In [1]:
# Practice Problems Nov. 12
# Zeke Van Dehy
# Nov 12, 2020

# Practice Problems Nov. 12 #

These problems are designed to make you practice working with files, looping, extracting values of interest from data, and putting values into different data structures. You should be able to solve all of them with techniques we have already learned. Take time to think about what it is you want out of the data.

# 1. Extract information from a GFF file #

This is a new file format that we have not seen yet, but it is very straightforward.

```NC_007898.3	RefSeq	CDS	40319	42571	.	-	0	ID=cds21;Parent=gene37;Dbxref=Genbank:YP_008563088.1;Name=YP_008563088.1;gbkey=CDS;gene=psaA;product=photosystem I P700 apoprotein A1;protein_id=YP_008563088.1;transl_table=11```

Each line is long and contains the following information:

```
[0] Database identifier
[1] Database
[2] Type of feature (in the example file these are all CDS, coding sequence)
[3] Start coordinate
[4] End coordinate
[5] placeholder (.)
[6] strandedness
[7] phase (reading frame -- how many bases have to be removed to get to the start of the next codon)
[8] information about the coding sequence```

The start coordinate and end coordinate refer to the position of the gene in a genomic sequence file like NC_007898.fasta, which you worked with back in the restriction enzyme exercise.

Your goal is to get enough information out of this file that you could use it to extract slices corresponding to gene locations from the .fasta file.

First, just open the file and write a loop that will loop over the lines and will get out the four fields that describe the gene position -- the start, stop, direction and phase. Store those values in a tuple, so that the end result is a list of tuples.

In [8]:
def getLocationInfo(filename):
    with open(filename) as fo:
        locationInfos = []
        for line in fo.readlines():
            line = line.strip()
            row = line.split("\t")
            start = int(row[3])
            stop = int(row[4])
            direction = row[6]
            phase = int(row[7])
            locationInfos.append((start,stop,direction,phase))
    return locationInfos

import pprint
pp = pprint.PrettyPrinter()

locations = getLocationInfo("NC_007898.cds.gff")
pp.pprint(("start","stop","direction","phase"))
pp.pprint(locations[:10])

('start', 'stop', 'direction', 'phase')
[(71636, 71749, '-', 0),
 (99801, 100032, '-', 0),
 (99239, 99264, '-', 2),
 (537, 1598, '-', 0),
 (2124, 3653, '-', 0),
 (6025, 6064, '-', 0),
 (4934, 5160, '-', 2),
 (7584, 7769, '+', 0),
 (8131, 8241, '+', 0),
 (10221, 11744, '-', 0)]


# 2. Use the coordinates from the GFF to extract sequence #

Now we can use the coordinates from the GFF to extract sequence from the FASTA file.  There are going to be two major steps here:

```1) open and read in the genomic FASTA file so that it's one big sequence```
```2) iterate over the list of tuples you made above. For every pair of coordinates, extract the sequence between them from the genomic FASTA sequence```

# Step 1 #

In the restriction enzyme exercise we did a few weeks ago, I gave you the code that would extract a single long sequence from a genomic FASTA file. Go back, find the correct code, and make any modifications that you think you need so that it will work for this problem.

In [14]:
def readFasta(filename):
    seqs = {}
    
    with open(filename) as fo:
        identifier = ""
        dna = ""
        for line in fo.readlines():
            line = line.strip()
            if line[0] == ">":
                identifier = line[1:]
                dna = ""
            else:
                dna += line
                seqs[identifier] = dna #would be more efficient to only do this when finish reading the sequence
    return seqs
DNASeq = readFasta("NC_007898.fasta")["NC_007898.3"]
print(DNASeq[:100])

TGGGCGAACGACGGGAATTGAACCCGCGCATGGTGGATTCACAATCCACTGCCTTGATCCACTTGGCTACATCCGCCCCCTCGCCTACTTACATTCCATT


# Step 2 #

Extract sequence slices that correspond to the pairs of coordinates. For now, don't worry about strand and phase, you just want to get the basic loop working. 

Remember that when looping over a tuple of four items, you would need to write your loop in the form:

```for a,b,c,d in listoftuples:```

And then you could work with the four values that make up the tuple directly as a, b, c and d.

Also remember that genomic coordinates start at 1, while string slices start at 0, so to get the correct coordinates you will need to offset the values.

In [20]:

for (start,stop,direction,phase) in locations[:10]:
    print(start,stop)
    geneSeq = DNASeq[start-1:stop-1]
    print(geneSeq)
    print(len(geneSeq))
    print("----------------")

71636 71749
ATACACCCTAGTACATGTTCCTCGACGCTGAGGGCACCCCCGAAGAGCGGGGGATTTCGTGACATTTCTGATTGGCTGTCTTGTATTTCTAATAAGTTGTTTAATAGTTGGCA
113
----------------
99801 100032
TAGAACGCCCTTGTTGACGATCCTTTACTCCGACAGCATCTAGGGTTCCTCGAACAATGTGATATCTCACACCGGGTAAATCCTTAACCCTTCCCCCTCTTACTAAGACTACAGAATGTTCTTGTGAATTATGGCCAATACCGGGTATATAAGCAGTGATTTCAAATCCAGAGGTTAATCGTACTCTGGCAACTTTACGTAAGGCAGAGTTTGGTTTTTTTGGGGTGATAG
231
----------------
99239 99264
TTATTTTGGCTTTTTTACCCCATAT
25
----------------
537 1598
TTATCCATTTGTAGATGGAGCTTCGATAGCAGCTAGGTCTAGAGGGAAGTTATGAGCATTACGTTCATGCATAACTTCCATACCAAGGTTAGCACGGTTGATGATATCAGCCCAAGTGTTAATTACACGACCCTGACTGTCAACTACAGATTGGTTGAAATTGAAACCATTTAGGTTGAAAGCCATAGTGCTGATACCTAAAGCGGTAAACCAGATACCTACTACAGGCCAAGCAGCTAGGAAGAAGTGTAACGAACGAGAGTTGTTGAAACTAGCATATTGGAAGATCAATCGGCCAAAATAACCATGAGCGGCTACGATATTATAAGTTTCTTCCTCTTGACCGAATCTGTAACCTTCATTAGCAGATTCATTTTCTGTGGTTTCCCTGATCAAACTAGAAGTTACCAAGGAACCATGCATAGCACTGAATAGGGAGCCGCCGAATACCCCAGCTACGCCTAACATGTGAAATGGGTGCATAAGGATGTTGTGTTCAGCCTGGAATACAATCATGAAA

# 3. Now, refine your solution #

If you've gotten this far, you've established that you can get a range of genomic DNA out of a file using coordinates extracted from a GFF file.

This is a very common bioinformatics task. NCBI Genbank distributes genomic sequence as a DNA fasta file + separate GFF file as one of its standard downloadable formats of genome information.

Ideally, you would like to get that sequence out and into more usable format, so you need to go back and add some refinements to your basic script.

# Extract more information about each gene #

Assume your end goal is to write out the individual sequences you extract into another standard bioinformatics format, like a multi-entry FASTA file. One thing you're going to need for that is an information line that tells something about what the sequence is. In the GFF file, that information is stored in the big information field at the end of the line. 

```NC_007898.3	RefSeq	CDS	40319	42571	.	-	0	ID=cds21;Parent=gene37;Dbxref=Genbank:YP_008563088.1;Name=YP_008563088.1;gbkey=CDS;gene=psaA;product=photosystem I P700 apoprotein A1;protein_id=YP_008563088.1;transl_table=11```

One of the subfields that you can extract starts with Dbxref=. This gives a Genbank database identifier for the particular gene. That would let a user cross-reference the sequence with other information. Another useful field that you can easily extract would be the one that starts with gene=. This gives a gene code that may be recognizable to the user. **If you do your line-splitting operations in the right order**, you could also extract the product= subfield, which has even more description about what the gene does.

Once you have gene=genename, you probably want to split it by the "=" so that you store only the actual gene name (or whatever) and not the generic label and equals sign. So that's a lot of line splits to practice.

Go back to the first loop you wrote that extracts the GFF information, and modify it so that you can extract these three fields **in addition** to the coordinates, strand and phase. 

Store these items in a dictionary, where the key is the unique Genbank identifier, and the value is a tuple of the rest of the information.

In [84]:
def getGeneInfo(filename):
    with open(filename) as fo:
        geneInfos = {}
        for line in fo.readlines():
            line = line.strip()
            row = line.split("\t")
            start = int(row[3])
            stop = int(row[4])
            direction = row[6]
            phase = int(row[7])
            db = ""
            gene = ""
            product = ""
            for chunk in row[8].split(";"):
                parts = chunk.split("=")
                if parts[0] == "Dbxref":
                    db = parts[1]
                if parts[0] == "gene":
                    gene = parts[1]
                if parts[0] == "product":
                    product = parts[1]
            if not db in geneInfos: 
                geneInfos[db] = (start,stop,direction,phase,gene,product)
#                 print(db, geneInfos[db])
    return geneInfos

geneInfos = getGeneInfo("NC_007898.cds.gff")

# pp.pprint(geneInfos)


# 4. Now go back and extract the sequences again #

Loop over the dictionary you made and go back to get the gene coordinates. Use the following method for your loop:

- Loop over sorted(dict.keys()) # this will sort the keys in order of Genbank ID
- For each key, use dict.get() to get the value
- Access the parts of the value tuple you need with their indexes

(You could do this several other ways, and if you like one of them a lot better for some reason, go ahead and solve it the way that works best for you.)

Once you can get the coordinates and other information from the value tuple, we can consider how we'd like to store the end result.

In [85]:
DNASeq = readFasta("NC_007898.fasta")["NC_007898.3"]
geneInfos = getGeneInfo("NC_007898.cds.gff")

def reverseComp(s):
    swaptable = str.maketrans("GCTA", "CGAT")
    return s.translate(swaptable)[::-1]

for key in sorted(geneInfos.keys()):
#     print(key)
    info = geneInfos.get(key)
    start,stop,direction,phase,gene,product = info
    
    geneSeq = DNASeq[start-1:stop-1]
    
    if direction == "-":
        geneSeq = reverseComp(geneSeq)
    geneSeq = geneSeq[phase:]
    
#     print(geneSeq)
    
    

# 5. Make the results more useful #

The end result that you want to have is a pair of items. A "name" that is made up of the descriptive information about the gene (Genbank ID, gene and product), and a "seq" that is made of the sequence saved **in the correct direction and phase**. 

Remember that if we have a gene sequence that is on the - strand of a genome, we really want its reverse and complement. So **if** strand == "-", you can use functions that we have made before to reverse and complement it. Similarly, **if** phase != 0, you need to slice off the first 1 or 2 (depending on the value of phase) sequence positions to get to the first codon.

"Name" should look something like Dbxref + " " + gene + " " + product, and seq should be a DNA sequence. Store these in a list of tuple pairs.

In [86]:
DNASeq = readFasta("NC_007898.fasta")["NC_007898.3"]
geneInfos = getGeneInfo("NC_007898.cds.gff")
result = []
for key in sorted(geneInfos.keys()):
    info = geneInfos.get(key)
    start,stop,direction,phase,gene,product = info
    
    geneSeq = DNASeq[start-1:stop-1]
    
    if direction == "-":
        geneSeq = reverseComp(geneSeq)
    geneSeq = geneSeq[phase:]
    
    result.append((key+" "+gene+" "+product+" "+geneSeq))
    
pp.pprint(result[:5])

['Genbank:YP_008563067.1 rps12 ribosomal protein S12 '
 'TGCCAACTATTAAACAACTTATTAGAAATACAAGACAGCCAATCAGAAATGTCACGAAATCCCCCGCTCTTCGGGGGTGCCCTCAGCGTCGAGGAACATGTACTAGGGTGTAT',
 'Genbank:YP_008563068.1 psbA photosystem II protein D1 '
 'TGACTGCAATTTTAGAGAGACGCGAAAGCGAAAGCCTATGGGGTCGCTTCTGTAACTGGATAACTAGCACTGAAAACCGTCTTTACATTGGATGGTTTGGTGTTTTGATGATCCCTACCTTATTGACGGCAACTTCTGTATTTATTATTGCCTTCATTGCTGCTCCTCCAGTAGACATTGATGGTATTCGTGAACCTGTTTCAGGGTCTCTACTTTACGGAAACAACATTATTTCCGGTGCCATTATTCCTACTTCTGCAGCTATAGGTTTACATTTTTACCCAATCTGGGAAGCGGCATCCGTTGATGAATGGTTATACAACGGTGGTCCTTATGAACTAATTGTTCTACACTTCTTACTTGGCGTAGCTTGTTACATGGGTCGTGAGTGGGAGCTTAGCTTCCGTCTGGGTATGCGACCTTGGATTGCTGTTGCATATTCAGCTCCTGTTGCAGCTGCTACCGCAGTTTTCTTGATCTACCCAATCGGTCAAGGAAGTTTTTCTGATGGTATGCCTCTAGGAATCTCTGGTACTTTCAATTTCATGATTGTATTCCAGGCTGAACACAACATCCTTATGCACCCATTTCACATGTTAGGCGTAGCTGGGGTATTCGGCGGCTCCCTATTCAGTGCTATGCATGGTTCCTTGGTAACTTCTAGTTTGATCAGGGAAACCACAGAAAATGAATCTGCTAATGAAGGTTACAGATTCGGTCAAGAGGAAGAAACTTATAATATCGTAGCCGCTCATGGTTATTTTGGC

# OPTIONAL / BROWNIE POINTS: Translate the DNA sequence to protein before storing #

We just did this with a dictionary. Can you include your translation routine in the script and make it work?

**Warning: this might tell you something about whether you really have your sequence in the correct reading frame. What result would you see that would make you need to go back and correct your reading frame slice calculations?**

In [87]:
def getTranslation(filename):
    codons = {}
    with open(filename) as fo:
        temp = {}
        for line in fo.readlines():
            line = line.strip()
            row = line.split("=")
            row[0] = row[0].strip()
            row[1] = row[1].strip()
            temp[row[0]] = row[1]
        
        for i, aa in enumerate(temp["AAs"]):
            c = temp["Base1"][i] + temp["Base2"][i] + temp["Base3"][i]
            codons[c] = aa
    return codons

def translateDNA(dna, translation):
    aalist = []
    for i in range(0,len(dna)-2,3):
        codon = dna[i:i+3]
        aalist.append(translation.get(codon,""))
    return "".join(aalist)

def generateProteinFasta(dnaFilename, aaFilename, codonFilename):
    tt = readCodonFile(codonFilename)
    dnaSeqs = readFasta(dnaFilename)
    toWrite = ""
    for identifier, dna in dnaSeqs.items():
        toWrite += ">"+identifier+"\n"
        toWrite += translateDNA(dna,tt)+"\n"
    f = open(aaFilename, "w")
    f.write(toWrite)
    f.close()

In [92]:
pp = pprint.PrettyPrinter(2)
DNASeq = readFasta("NC_007898.fasta")["NC_007898.3"]
geneInfos = getGeneInfo("NC_007898.cds.gff")

transTable = getTranslation("bacterialcode.txt") #NCBI Trans Table 11

result = []

for key in sorted(geneInfos.keys()):
    
    info = geneInfos.get(key)
    start,stop,direction,phase,gene,product = info
    
    geneSeq = DNASeq[start-1:stop]
    
    if direction == "-":
        geneSeq = reverseComp(geneSeq)
    geneSeq = geneSeq[phase:]
    
    protein = translateDNA(geneSeq,transTable)
    
    result.append((key+" "+gene+" "+product+" "+protein))
    
pp.pprint(result[:5])

[ 'Genbank:YP_008563067.1 rps12 ribosomal protein S12 '
  'MPTIKQLIRNTRQPIRNVTKSPALRGCPQRRGTCTRVY',
  'Genbank:YP_008563068.1 psbA photosystem II protein D1 '
  'MTAILERRESESLWGRFCNWITSTENRLYIGWFGVLMIPTLLTATSVFIIAFIAAPPVDIDGIREPVSGSLLYGNNIISGAIIPTSAAIGLHFYPIWEAASVDEWLYNGGPYELIVLHFLLGVACYMGREWELSFRLGMRPWIAVAYSAPVAAATAVFLIYPIGQGSFSDGMPLGISGTFNFMIVFQAEHNILMHPFHMLGVAGVFGGSLFSAMHGSLVTSSLIRETTENESANEGYRFGQEEETYNIVAAHGYFGRLIFQYASFNNSRSLHFFLAAWPVVGIWFTALGISTMAFNLNGFNFNQSVVDSQGRVINTWADIINRANLGMEVMHERNAHNFPLDLAAIEAPSTNG*',
  'Genbank:YP_008563069.1 matK maturase K '
  'MEEIHRYLQPDSSQQHNFLYPLIFQEYIYALAQDHGLNRNRSILLENSGYNNKFSFLIVKRLITRMDQQNHLIISTNDSNKNPFLGCNKSLYSQMISEGFACIVEIPFSIRLISSLSSFEGKKIFKSHNLRSIHSTFPFLEDNFSHLNYVLDILIPYPVHLEILVQTLRYWVKDASSLHLLRFFLHEYCNLNSLITSKKPGYSFSKKNQRFFFFLYNSYVYECESTFVFLRNQSSHLRSTSFGALLERIYFYGKIERLVEAFAKDFQVTLWLFKDPVMHYVRYEGKSILASKGTFPWMNKWKFYLVNFWQCHFSMYFNTGRIHINQLSNHSRDFMGYLSSVRLNHSMVRSQMLENSFLINNPIKKFDTLVPIIPLIGSLAKAHFCTGLGHPISKPVWSDLSDSDIIDRFGRICRNLFHYYSGSSKKKTLYRIKY

# OPTIONAL / BROWNIE POINTS: Write the names and seqs out to a FASTA format file #

A FASTA file looks like:

> name
SEQSEQSEQSEQSEQSEQetc

Loop over your list of tuples and write your sequences out to a FASTA format file. The sequence lines can be however long they are.



In [98]:
DNASeq = readFasta("NC_007898.fasta")["NC_007898.3"]
geneInfos = getGeneInfo("NC_007898.cds.gff")
result = ""
for key in sorted(geneInfos.keys()):
    info = geneInfos.get(key)
    start,stop,direction,phase,gene,product = info
    
    geneSeq = DNASeq[start-1:stop-1]
    
    if direction == "-":
        geneSeq = reverseComp(geneSeq)
    geneSeq = geneSeq[phase:]
    
    result += ">"+str(key)+"\n"
    result += geneSeq+"\n"
    
with open("practice.fasta","w") as fo:
    fo.write(result)

# OPTIONAL / BROWNIE POINTS: Write the sequence in 80 character lines #

A really standard FASTA format breaks the sequence up into 80 character lines, for legacy reasons that have to do with how old-school programming languages like FORTRAN work. You can do this using a range with a step looping over the sequence...see if you can figure it out. I'll show you how next week.

In [100]:
DNASeq = readFasta("NC_007898.fasta")["NC_007898.3"]
geneInfos = getGeneInfo("NC_007898.cds.gff")
result = ""
for key in sorted(geneInfos.keys()):
    info = geneInfos.get(key)
    start,stop,direction,phase,gene,product = info
    
    geneSeq = DNASeq[start-1:stop-1]
    
    if direction == "-":
        geneSeq = reverseComp(geneSeq)
    geneSeq = geneSeq[phase:]
    
    result += ">"+str(key)+"\n"
    for i in range(0,len(geneSeq),80):
        result += geneSeq[i:i+80]+"\n"
    
with open("practice80.fasta","w") as fo:
    fo.write(result)